# Data
---

In [1]:
import os
import json
import pickle
import random

from tqdm.auto import tqdm
from valerie.utils import get_logger
from valerie.preprocessing import extract_words_from_url, clean_text
from valerie.scoring import validate_predictions_phase2, compute_score_phase2
from valerie.modeling import SequenceClassificationModel, SequenceClassificationDataset, SequenceClassificationExample

In [2]:
_logger = get_logger()

In [3]:
with open("data/phase2-validation/processed/responses.pkl", "rb") as fi:
    responses = pickle.load(fi)

In [ ]:
# responses = random.sample(responses, k=100)

In [13]:
def compute_responses_score(results, claims_dict):
    predictions = {}
    perfect_predictions = {}
    labels = {}

    for k, hits in results.items():
        claim = claims_dict[k]
        labels[claim.id] = claim.to_dict()
        
        hits = sorted(hits, key=lambda x: x[1], reverse=True) # sort by score
        predictions[claim.id] = {
            "label": claim.label,
            "explanation": "",
            "related_articles": {
                i + 1: x
                for i, x in enumerate([v[0] for v in hits[:2]])
            }
        }
        perfect_predictions[claim.id] = {
            "label": claim.label,
            "explanation": "",
            "related_articles": {
                i + 1: x
                for i, x in enumerate([v[0] for v in hits if v[0] in claim.related_articles.values()][:2])
            }
        }

    validate_predictions_phase2(predictions)
    score = compute_score_phase2(labels, predictions)
    validate_predictions_phase2(perfect_predictions)
    perfect_score = compute_score_phase2(labels, perfect_predictions)
    return {
        "perfect_rerank_score": perfect_score["score"],
        "perfect_rerank_error": perfect_score["error"],
        "api_score": score["score"],
        "api_error": score["error"],
    }

# Examples
---

In [5]:
def create_text_a(claim):
    text_a = claim.claim
    text_a += " "
    text_a += claim.claimant if claim.claimant else "no claimant"
    text_a += " "
    text_a += claim.date.split()[0].split("T")[0] if claim.date else "no date"
    return clean_text(text_a)

def create_text_b(article):
    text_b = ""
    if article.source:
        text_b += article.source + ". "
    if article.title:
        text_b += article.title + ". "
    if article.url:
        url_words = extract_words_from_url(article.url)
        if url_words:
            text_b += " ".join(url_words) + ". "
    if article.content:
        text_b += article.content
    return clean_text(text_b)

misses = 0
examples = []
for res in tqdm(responses):
    claim = res["claim"]
    if not res["res"]:
        misses += 1
        continue
    hits = [(hit["score"], hit["article"]) for hit in res["res"]["hits"]["hits"]]
    
    text_a = create_text_a(claim)
    
    related_articles_url_set = set(claim.related_articles.values())
    
    for score, article in hits:
        text_b = create_text_b(article)
            
        examples.append(SequenceClassificationExample(
            guid=claim.index,
            text_a=text_a,
            text_b=text_b,
            label=1 if article.url in related_articles_url_set else 0,
            art_id=article.index
        ))

In [7]:
examples[0]

{
  "guid": "Phase2ValidationDataset/383",
  "text_a": "\"Huge! Results From Breaking Chloroquine Study Show 100% Cure Rate For Patients Infected With The Coronavirus.\" Facebook post 2020-03-19",
  "text_b": "teaparty. tea party huge results from breaking chloroquine study show cure rate for patients infected with the coronavirus. huge! results from breaking chloroquine study show 100% cure rate for patients infected with the coronavirus | tea party (gateway pundit) \u2013 on monday dr. anthony fauci, director of the national institute of allergy and infectious diseases, announced that the first trial vaccine for the coronavirus is now being tested. the trial taking place in seattle, which has been a hotbed for covid-19. the test includes 45 people age 18-55 and they are receiving two injections, one at zero days, one at 28 days. the individuals will then be followed for one year. the trial results is still months away. on monday night laura ingraham reported that a new study revealed

In [8]:
misses

0

In [9]:
print(len(responses)*30)
print(len(examples))

15000
14935


# Predict
---

In [10]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "dryrun" 
os.environ["WANDB_WATCH"] = "false" 

In [14]:
# def run_a_model(examples, pretrained_model_name_or_path, nproc=4, predict_batch_size=8):

# "castorini/monot5-base-msmarco"
# "castorini/monobert-large-msmarco"
# "nboost/pt-bert-large-msmarco"]:
pretrained_model_name_or_path = "castorini/monobert-large-msmarco"
nproc=16
predict_batch_size=64

print(pretrained_model_name_or_path.center(80, "-"))

model = SequenceClassificationModel.from_pretrained(pretrained_model_name_or_path)
examples_dataset = model.create_dataset(examples, nproc=nproc)
predict_output = model.predict(examples_dataset, predict_batch_size=predict_batch_size)

claims_dict = {res["claim"].index: res["claim"] for res in responses if res["res"]}
api_scores_dict = {
    res["claim"].index: {
        hit["article"].index: hit["score"] for hit in res["res"]["hits"]["hits"]
    }
    for res in responses
    if res["res"]
}


rerank_just_api_responses = {
    res["claim"].index: [
        (hit["article"].index, hit["score"]) for hit in res["res"]["hits"]["hits"]
    ]
    for res in responses
    if res["res"]
}

rerank_just_trans_responses = {res["claim"].index: [] for res in responses if res["res"]}

rerank_both_responses = {
    res["claim"].index: []
    for res in responses
    if res["res"]
}

for example, proba in tqdm(zip(examples, predict_output.predictions)):
    proba = float(proba[1]) # get probability that the article is related

    rerank_just_trans_responses[example.guid].append((example.art_id, proba))
    rerank_both_responses[example.guid].append((example.art_id, proba + api_scores_dict[example.guid][example.art_id]))
    
print('api')
print(json.dumps(compute_responses_score(rerank_just_api_responses, claims_dict), indent=2))
print()
print('trans')
print(json.dumps(compute_responses_score(rerank_just_trans_responses, claims_dict), indent=2))
print()
print('both')
print(json.dumps(compute_responses_score(rerank_both_responses, claims_dict), indent=2))
print()
print()
print()

api
{
  "perfect_rerank_score": 1.1557801493610302,
  "perfect_rerank_error": "'None'",
  "api_score": 0.6617083840074162,
  "api_error": "'None'"
}

trans
{
  "perfect_rerank_score": 1.1557801493610302,
  "perfect_rerank_error": "'None'",
  "api_score": 0.7067266242656328,
  "api_error": "'None'"
}

both
{
  "perfect_rerank_score": 1.1557801493610302,
  "perfect_rerank_error": "'None'",
  "api_score": 0.698095691479061,
  "api_error": "'None'"
}





# Results 500
---

In [15]:
compute_responses_score(rerank_just_api_responses, claims_dict)

{'perfect_rerank_score': 1.1557801493610302,
 'perfect_rerank_error': "'None'",
 'api_score': 0.6617083840074162,
 'api_error': "'None'"}

In [16]:
compute_responses_score(rerank_just_trans_responses, claims_dict)

{'perfect_rerank_score': 1.1557801493610302,
 'perfect_rerank_error': "'None'",
 'api_score': 0.7067266242656328,
 'api_error': "'None'"}

In [17]:
compute_responses_score(rerank_both_responses, claims_dict)

{'perfect_rerank_score': 1.1557801493610302,
 'perfect_rerank_error': "'None'",
 'api_score': 0.698095691479061,
 'api_error': "'None'"}